In [1]:
import modelbit
mb = modelbit.login()

In [6]:
import warnings
warnings.filterwarnings('ignore')
import torch
from torch import nn
from torchvision import transforms
import cv2
import numpy as np
from PIL import Image
from io import BytesIO
import base64

def final_detection(frame_base64):
    frame_data = base64.b64decode(frame_base64)
    frame = Image.open(BytesIO(frame_data))

    im_size = 224
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    transform = transforms.Compose([
        transforms.Resize((im_size, im_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
    frame = transform(frame).unsqueeze(0)
    
    model_path = "/Users/karthiksagar/DeepFake-Detection/saved_best_model/best-checkpoint-model.pt"

    model = Model(2)

    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    
    model.eval()

    with torch.no_grad():
        frame_tensor = frame.cuda()
        logits = model(frame_tensor)
        sm = nn.Softmax(dim=1)
        probabilities = sm(logits)
        confidence, prediction = torch.max(probabilities, dim=1)    

    result = {
        "status": "real" if prediction.item() == 1 else "fake",
        "confidence": confidence.item() * 100
    }

    return result

In [48]:
mb.deploy(final_detection, extra_files={"/Users/karthiksagar/DeepFake-Detection/saved_best_model/best-checkpoint-model.pt" : "best_checkpoint_model.pt", "Model.py" : "Model.py"})

### PREDICTION USING BASE-64

In [2]:
import base64
from PIL import Image
import io

def image_to_base64(image_path):
    # Open the image file
    with Image.open(image_path) as image:
        # Convert image to byte stream
        buffered = io.BytesIO()
        image.save(buffered, format=image.format)
        img_bytes = buffered.getvalue()

        # Encode byte stream to base64
        img_base64 = base64.b64encode(img_bytes).decode('utf-8')

    return img_base64

# Example usage
image_path = 'F_FAP1_00572-3.png'
base64_string = image_to_base64(image_path)
print(base64_string)


...


In [7]:
print(final_detection(base64_string))

AttributeError: Can't get attribute 'Model' on <module '__main__'>

In [53]:
import warnings
warnings.filterwarnings('ignore')
import torch
from torch import nn
from torchvision import transforms
import cv2
import numpy as np
from PIL import Image
from io import BytesIO
import base64

def final(frame_base64):
    frame_data = base64.b64decode(frame_base64)
    frame = Image.open(BytesIO(frame_data))

    im_size = 224
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    transform = transforms.Compose([
        transforms.Resize((im_size, im_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
    frame = transform(frame).unsqueeze(0)
    
    model_path = "deepfake_model.pkl"

    model = torch.load(model_path)
    
    model.eval()

    with torch.no_grad():
        frame_tensor = frame.cuda()
        logits = model(frame_tensor)
        sm = nn.Softmax(dim=1)
        probabilities = sm(logits)
        confidence, prediction = torch.max(probabilities, dim=1)    

    result = {
        "status": "real" if prediction.item() == 1 else "fake",
        "confidence": confidence.item() * 100
    }

    return result

In [54]:
mb.deploy(final, extra_files={"/Users/karthiksagar/DeepFake-Detection/saved_best_model/deepfake_model.pkl" : "deepfake_model.pkl"})

Uploading '/Users/karthiksagar/DeepFake-Detection/saved_best_model/deepfake_model.pkl': 100%|██████████| 208M/208M [03:08<00:00, 1.10MB/s] 


## SAVING

In [22]:
class Model(nn.Module):
    def __init__(self, num_classes=2, latent_dim=2048, lstm_layers=1, hidden_dim=2048, bidirectional=False):
        super(Model, self).__init__()
        model = models.resnext50_32x4d(pretrained=True)  # Residual Network CNN
        self.model = nn.Sequential(*list(model.children())[:-2])
        self.lstm = nn.LSTM(latent_dim, hidden_dim, lstm_layers, bidirectional)
        self.relu = nn.LeakyReLU()
        self.dp = nn.Dropout(0.4)
        self.linear1 = nn.Linear(2048, num_classes)
        self.avgpool = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        x = self.model(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        return self.dp(self.linear1(x))

In [23]:
model = Model(2)
model_path = "/Users/karthiksagar/DeepFake-Detection/saved_best_model/best-checkpoint-model.pt"
torch.save(model, model_path)